In [ ]:
--Топ 5 пользователей по заказам за конкретный месяц(август):
SELECT user_id, COUNT(order_id) AS created_orders
FROM user_actions
WHERE DATE_PART('month', time) = 8.00
GROUP BY user_id, action
HAVING action = 'create_order'
ORDER BY created_orders DESC, user_id
LIMIT 5

--Разбиваем заказы по числу товаров в них на группы и выводим результат:
SELECT CASE
       WHEN array_length(product_ids, 1) <=3 THEN 'Малый'
       WHEN array_length(product_ids, 1) BETWEEN 4 and 6 THEN 'Средний'
       WHEN array_length(product_ids, 1) >=7 THEN 'Большой'
       END AS order_size, COUNT(order_id) AS orders_count
FROM orders
GROUP BY order_size
ORDER BY orders_count

--Среднее кол-во заказов для всех пользователей:
SELECT ROUND(AVG(count), 2)
FROM (SELECT COUNT(order_id), user_id
FROM user_actions GROUP BY user_id) as sub1

--Найти юзеров, сделавших заказ за последнюю неделю: 
SELECT count(distinct user_id) as users_count
FROM   user_actions
WHERE  time >= (SELECT max(time)
                FROM   user_actions) --ИЛИ NOW() - interval '1 week'

--ОТБОР собранных и не отмененных заказов:
WITH canceled as (
    SELECT order_id FROM user_actions
    WHERE action = 'cancel_order'
)
SELECT order_id from user_actions
WHERE order_id not IN ( SELECT * from canceled)
LIMIT 50

--Проверка заказов, которые приняты курьерами но не созданы пользователями.
WITH error as (SELECT order_id
             FROM   user_actions)
SELECT count(order_id) as orders_count
FROM   courier_actions
WHERE  order_id not in (SELECT *
                        FROM   error)
-- Отбираем отмененные, но доставленные заказы:
SELECT count(distinct order_id) as orders_canceled,
       count(order_id) filter (WHERE action = 'deliver_order') as orders_canceled_and_delivered
FROM   courier_actions
WHERE  order_id in (SELECT order_id
                    FROM   user_actions
                    WHERE  action = 'cancel_order')

--Выбираем 10 самых популярных товаров из неотмененных заказов и подсчитываем количество их покупок:
WITH mostpop AS (
    SELECT unnest(product_ids) AS product_id FROM orders
WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')),
    purchased AS (SELECT COUNT(product_id) AS times_purchased, product_id FROM mostpop GROUP BY product_id ORDER BY times_purchased DESC LIMIT 10) 
    SELECT * FROM purchased ORDER BY product_id

--Получаем выжимку из таблиц по количеству заработанных денег и количеству заказов с разбивкой по полу пользователей: 
SELECT COUNT(DISTINCT order_id) as orders, sex, sum(price) as total_revenue FROM 
(SELECT courier_actions.order_id, t1.product_id, products.price, products.name, t2.user_id, users.birth_date, users.sex
FROM courier_actions
LEFT JOIN (SELECT order_id, UNNEST(product_ids) AS product_id FROM orders)t1
ON courier_actions.order_id=t1.order_id
LEFT JOIN products
on t1.product_id = products.product_id
LEFT JOIN (SELECT DISTINCT user_id, order_id from user_actions)t2
ON courier_actions.order_id = t2.order_id
LEFT JOIN users
ON t2.user_id = users.user_id
WHERE courier_actions.action = 'deliver_order')t3
GROUP BY sex

--Количество заказов пользователя в прошлый его активный день:
SELECT user_id, count(order_id) AS daily_orders, time::DATE as date, rank() OVER(PARTITION BY user_id ORDER BY time::DATE)active_day_num,
LAG(count(order_id), 1) OVER(PARTITION BY user_id ORDER BY time::DATE) prev_day_orders
FROM user_actions
GROUP BY user_id, date
LIMIT 50

--Порядковые номера заказов каждого пользователя и интервал по времени между заказами для каждого пользователя:
SELECT user_id, order_id, time, ROW_NUMBER() OVER(Partition by user_id order by time)order_number, LAG(time, 1) OVER(Partition by user_id order by time)time_lag, time-(LAG(time, 1) OVER(Partition by user_id order by time))time_diff
from user_actions
WHERE order_id NOT IN(SELECT order_id From user_actions Where action = 'cancel_order')
Order by user_id, order_number
LIMIT 1000


--Количество часов между заказами для пользователя с более чем 1 заказом:
with hoursinterval as(SELECT user_id,
                  extract(epoch
           FROM   (time-lag(time, 1)
           OVER(
           PARTITION BY user_id
           ORDER BY time)))/3600 as hoursbetween
           FROM   user_actions
           WHERE  order_id not in(SELECT order_id
                                  FROM   user_actions
                                  WHERE  action = 'cancel_order')
           ORDER BY user_id)
SELECT user_id,
       round(avg(hoursbetween))::integer hours_between_orders
FROM  hoursinterval
WHERE  hoursbetween is not null
GROUP BY user_id
ORDER BY user_id limit 1000


--Среднее количество заказов, доставленное каждым курьером за месяц и проверка,  выше ли его активность, чем в среднем у курьеров или нет:
WITH s1 AS (SELECT courier_id, COUNT(DISTINCT order_id)delivered_orders
FROM courier_actions
WHERE action = 'deliver_order' and DATE_PART('month', time::DATE) = 9.00
GROUP BY courier_id)
SELECT courier_id, delivered_orders, ROUND(AVG(delivered_orders), 2) OVER()avg_delivered_orders, 
CASE WHEN delivered_orders/AVG(delivered_orders) OVER() >1 THEN 1
WHEN delivered_orders/AVG(delivered_orders) OVER() <1 THEN 0
END AS is_above_avg
FROM s1
ORDER BY courier_id


--Определяем количество первых и повторных заказов на каждую дату: 
With type_of_order as (SELECT CASE
WHEN ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY time) = 1 THEN 'Первый'
WHEN ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY time) != 1 THEN 'Повторный'
END AS order_type, DATE_trunc('day', time) date, order_id
FROM user_actions
WHERE order_id not in (SELECT order_id from user_actions WHERE action = 'cancel_order'))
SELECT order_type, date, COUNT(*)orders_count
FROM type_of_order
GROUP BY date, order_type
ORDER BY date, order_type

--Выводим данные о количестве сделанных и отмененных заказов для каждого пользователя на момент совершения нового действия и подсчет доли отмененных/успешных заказов:
WITH created_and_canceled_orders AS(SELECT user_id, order_id, time, action, count(order_id) filter (WHERE action = 'create_order') OVER (PARTITION BY user_id order by time)created_orders, count(order_id) filter (WHERE action = 'cancel_order') OVER (PARTITION BY user_id order by time)canceled_orders
FROM user_actions)
SELECT user_id, order_id, time, action, created_orders, canceled_orders, ROUND(canceled_orders::DECIMAL/created_orders, 2) AS cancel_rate
FROM created_and_canceled_orders
ORDER BY user_id, order_id, time
LIMIT 200
